# Introduction

In this notebook we demonstrate the use of **BM25 (Best Matching 25)** Information Retrieval technique to make trace link recovery between Use Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each use case content as an entire document that must be returned to the query made


## Import Libraries

In [109]:
import pandas as pd
import numpy as np
import seaborn as sns
from gensim.summarization.bm25 import BM25

import matplotlib.pyplot as plt

from sklearn.metrics import precision_recall_fscore_support, pairwise
from sklearn.externals.joblib import Parallel, delayed

from scipy.sparse import csr_matrix

import nltk
import datetime
import pprint
from enum import Enum
import pickle

import warnings; warnings.simplefilter('ignore')

### Oracle Loader

In [110]:
class OracleLoader:
    def __init__(self, rows_names, columns_names):
        self.oracle = None
        self._columns_names = columns_names
        self._rows_names = rows_names
    
    def load(self):
        self.oracle = pd.DataFrame(columns=list(self._columns_names), 
                                   data=np.zeros(shape=(len(self._rows_names), len(self._columns_names)), 
                                                 dtype='int64'))
        self.oracle.insert(0, 'artf_name', list(self._rows_names))
        
        for index, row in trace_df.iterrows():
            idx = self.oracle[self.oracle.artf_name == row['trg_artf']].index
            self.oracle.at[idx, row['src_artf']] = row['link']

        self.oracle.set_index('artf_name', inplace=True)

## Load Dataset and Preprocessing

In [111]:
trace_df = pd.read_csv('../../data/jEdit/jEditDataset/oracle/output/trace_matrix.csv')
artfs_desc_df = pd.read_csv('../../data/jEdit/jEditDataset/oracle/output/artifacts_descriptions.csv', sep="|")

use_cases_df = artfs_desc_df[artfs_desc_df.artf_description.str.contains('Use Case ID')]
bug_reports_df = artfs_desc_df[artfs_desc_df.artf_description.str.contains('Bug Number')]

corpus = use_cases_df.artf_description
query = bug_reports_df.artf_description

use_cases_names = use_cases_df.artf_name
bug_reports_names = bug_reports_df.artf_name

orc = OracleLoader(use_cases_names, bug_reports_names)
orc.load()

# BM25 Model

#### Model Hyperparameters

In [112]:
class BM25_Model_Hyperp(Enum):
    NAME = 'bm25__name'
    TOP = 'bm25_top'
    K = 'bm25__k'
    B = 'bm25__b'
    EPSILON = 'bm25__epsilon'
    TOKENIZER = 'bm25__tokenizer'
    SIM_MEASURE_MIN_THRESHOLD = 'bm25__sim_measure_min_threshold'

#### Tokenizers

In [113]:
"""
Others stemmers are not relevant for our analysis:
 . RSLP Stemmer: portuguese language
 . ISRIS Stemmer: returns Arabic root for the given token 
 . Regexp Stemmer: uses regulax expressions to identify morphological affixes
 
Relevant Stemmers/Lemmatizers are implemented below. 
"""

class GenericTokenizer(object):
    def __init__(self):
        self.stopwords = nltk.corpus.stopwords.words('english')
    def __call__(self, doc):
        tokens = [self.stemmer.stem(token) for token in nltk.word_tokenize(doc)]
        #return [token.lower() for token in tokens if token.isalpha() and token not in self.stopwords and len(token) > 1]
        return [token.lower() for token in tokens if token not in self.stopwords]
        
class WordNetBased_LemmaTokenizer(GenericTokenizer):
    def __init__(self):
        super().__init__()
        self.wnl = nltk.stem.WordNetLemmatizer()
    def __call__(self, doc):
        tokens = [self.wnl.lemmatize(token) for token in nltk.word_tokenize(doc)]
        return [token.lower() for token in tokens if token.isalpha() and token not in self.stopwords]

class LancasterStemmerBased_Tokenizer(GenericTokenizer):
    def __init__(self):
        super().__init__()
        self.stemmer = nltk.stem.LancasterStemmer()
    def __call__(self, doc):
        return super().__call__(doc)

class PorterStemmerBased_Tokenizer(GenericTokenizer):
    def __init__(self):
        super().__init__()
        self.stemmer = nltk.stem.PorterStemmer()
    def __call__(self, doc):
        return super().__call__(doc)
    
class SnowballStemmerBased_Tokenizer(GenericTokenizer):    
    def __init__(self):
        super().__init__()
        self.stemmer = nltk.stem.SnowballStemmer('english')    
    def __call__(self, doc):
        return super().__call__(doc)
        

#### Quick Test with Model

In [114]:
from gensim.summarization.bm25 import BM25

corpus__ = [
     ["black", "cat", "white", "cat", "blacks"],
     ["cat", "outer", "space"],
     ["wag", "dog"],
     ['blacks']
 ]

queries = [['black'], ['cat']]

bm25__ = BM25(corpus__)

average_idf = sum(map(lambda k: float(bm25__.idf[k]), bm25__.idf.keys())) / len(bm25__.idf.keys())

result_1 = bm25__.get_scores(queries[0], average_idf=average_idf)
result_2 = bm25__.get_scores(queries[1], average_idf=average_idf)

print('result_1: {}'.format(result_1))
print('result_2: {}'.format(result_2))

result_1: [0.6192874727082551, 0, 0, 0]
result_2: [0.0, 0.0, 0, 0]


#### Model Defintion

In [118]:
"""
params_dict = {
    'bm25__k' : 1.2,
    'bm25__b' : 0.75,
    'bm25__epsilon' : 0.25,
    'bm25__name' : 'BM25',
    'bm25__tokenizer' : Tokenizer(),
    'bm25__min_threshold' : 3
}
"""
class BM_25:
    # k = 1.2, b = 0.75, coord_factor = False
    def __init__(self, **kwargs):
        self.k = None
        self.b = None
        self.epsilon = None
        self.name = None
        self.top = None
        self.sim_measure_min_threshold = None
        self.tokenizer = None
        self.trace_links_df = None
        
        self._sim_matrix = None
        
        self.set_basic_params(**kwargs)
        self.set_tokenizer(**kwargs)
    
    
    def set_basic_params(self, **kwargs):
        self.name = 'BM25' if BM25_Model_Hyperp.NAME.value not in kwargs.keys() else kwargs[BM25_Model_Hyperp.NAME.value]
        self.k = 1.2 if BM25_Model_Hyperp.K.value not in kwargs.keys() else kwargs[BM25_Model_Hyperp.K.value]
        self.b = 0.75 if BM25_Model_Hyperp.B.value not in kwargs.keys() else kwargs[BM25_Model_Hyperp.B.value]
        self.epsilon = 0.25 if BM25_Model_Hyperp.EPSILON.value not in kwargs.keys() else kwargs[BM25_Model_Hyperp.EPSILON.value]
        self.top = 3 if BM25_Model_Hyperp.TOP.value not in kwargs.keys() else kwargs[BM25_Model_Hyperp.TOP.value]
        self.sim_measure_min_threshold = ('', 0.0) if BM25_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value not in kwargs.keys() else kwargs[BM25_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value]
        
    def set_tokenizer(self, **kwargs):
        self.tokenizer = WordNetBased_LemmaTokenizer() if BM25_Model_Hyperp.TOKENIZER.value not in kwargs.keys() else kwargs[BM25_Model_Hyperp.TOKENIZER.value]
        
        #tokenizer_params = {key.split('__')[2]:kwargs[key] for key,val in kwargs.items() if '__tokenizer__' in key}
        #self.tokenizer.set_params(**tokenizer_params)
    
    def set_name(self, name):
        self.name = name
        
    def recover_links(self, corpus, query, use_cases_names, bug_reports_names):
        bm25 = BM25([self.tokenizer.__call__(doc) for doc in corpus])
        average_idf = sum(map(lambda k: float(bm25.idf[k]), bm25.idf.keys())) / len(bm25.idf.keys())
        query = [self.tokenizer.__call__(doc) for doc in query]
        
        self._sim_matrix = pd.DataFrame(index = use_cases_names, 
                                           columns = bug_reports_names,
                                           data=np.zeros(shape=(len(use_cases_names), len(bug_reports_names)),dtype='float64'))
        
        for bug_id, bug_desc in zip(bug_reports_names, query):
            scores = bm25.get_scores(bug_desc, average_idf=average_idf)
            for uc_id, sc in zip(use_cases_names, scores):
                self._sim_matrix.at[uc_id, bug_id] = sc
        
        self._fillUp_traceLinksDf(use_cases_names, bug_reports_names, self._sim_matrix)
    
    def _fillUp_traceLinksDf(self, use_cases_names, bug_reports_names, sim_matrix):
        self.trace_links_df = pd.DataFrame(index = use_cases_names,
                                           columns = bug_reports_names,
                                           data = sim_matrix)
                    
        for col in self.trace_links_df.columns:
            nlargest_df = self.trace_links_df.nlargest(n = self.top, columns=col, keep='first')    
            self.trace_links_df[col] = [1 if x in nlargest_df[col].tolist() and x >= self.sim_measure_min_threshold[1] else 0 for x in self.trace_links_df[col]]
    
    def model_setup(self):
        return {"Setup" : 
                  [
                      {"Name" : self.name},
                      {"Top Value" : self.top},
                      {"Sim Measure Min Threshold" : self.sim_measure_min_threshold},
                      {"K" : self.k},
                      {"B" : self.b},
                      {"Epsilon" : self.epsilon},
                      {"Tokenizer Type" : type(self.tokenizer)}
                  ]
               }
    
    def get_name(self):
        return self.name
    
    def get_top_value(self):
        return self.top
    
    def get_sim_matrix(self):
        return self._sim_matrix
    
    def get_tokenizer_type(self):
        return type(self.tokenizer)
    
    def get_trace_links_df(self):
        return self.trace_links_df

### Model Evaluator

In [116]:
class ModelEvaluator:
    def __init__(self, oracle, model):
        self.model = model
        self.oracle = oracle
        self.recovered_links = model.trace_links_df
        
        self.eval_df = pd.DataFrame(columns=['precision','recall','fscore','support'])
        self.mean_precision = -1
        self.mean_recall = -1
        self.mean_fscore = -1
    
    def evaluate_model(self, verbose=False, file=None):
        y_true = csr_matrix(self.oracle.values, dtype=int)
        y_pred = csr_matrix(self.recovered_links.values, dtype=int)
        
        p, r, f, sp = precision_recall_fscore_support(y_true, y_pred)

        i = 0
        for idx, row in self.oracle.iteritems():
            self.eval_df.at[idx, 'precision'] = p[i]
            self.eval_df.at[idx, 'recall'] = r[i]
            self.eval_df.at[idx, 'fscore'] = f[i]
            self.eval_df.at[idx, 'support'] = sp[i]
            i += 1
        
        self.mean_precision = self.eval_df.precision.mean()
        self.mean_recall = self.eval_df.recall.mean()
        self.mean_fscore = self.eval_df.fscore.mean()
        
        if verbose:
            self.print_report(file)
    
    #def check_best_model(self, best_pre, best_rec, best_fs, best_md):
    #    if best_rec <= self.get_mean_recall():
    #        if best_pre <= self.get_mean_precision():
    #            return (self.get_mean_precision(), self.get_mean_recall(), self.get_mean_fscore(), self.get_model())
    #    return (best_pre, best_rec, best_fs, best_md)
    
    def print_report(self, file=None):
        dic = self.model.model_setup()
        dic['Measures'] = {}
        dic['Measures']['Mean Precision of {}'.format(self.model.get_name())] = self.get_mean_precision()
        dic['Measures']['Mean Recall of {}'.format(self.model.get_name())] = self.get_mean_recall()
        dic['Measures']['Mean FScore of {}'.format(self.model.get_name())] = self.get_mean_fscore()
        
        if file is None:    
            pprint.pprint(dic)
        else:
            file.write(pprint.pformat(dic))
        
    def plot_precision_vs_recall(self):
        plt.figure(figsize=(6,6))
        plt.plot(self.eval_df.recall, self.eval_df.precision, 'ro', label='Precision vs Recall')

        plt.ylabel('Precision')
        plt.xlabel('Recall')

        plt.axis([0, 1.1, 0, 1.1])
        plt.title("Precision vs Recall Plot - " + self.model.get_name())
        plt.show()
    
    def save_log(self):
        print("\nSaving model log...")
        with open('../logs/' + str(datetime.datetime.now()) + '.txt', 'a') as f:
            evaluator.evaluate_model(verbose=True, file=f)
        print("Model log saved with success!")
            
    def get_mean_precision(self):
        return self.mean_precision
    
    def get_mean_recall(self):
        return self.mean_recall
    
    def get_mean_fscore(self):
        return self.mean_fscore

    def get_model(self):
        return self.model

## Evaluate Recovering Efficiency

In order to evaluate the efficiency of the algorithm tested (LSI), we use common metrics applied in the field of IR:

    * Precision
    * Recall
    * F1-score

#### Analysis with Default Values of BM25 Model

In [119]:
best_model = BM_25()
best_model.recover_links(corpus, query, use_cases_names, bug_reports_names)

best_model.get_sim_matrix()

#evaluator = ModelEvaluator(orc.oracle, best_model)
#evaluator.evaluate_model(verbose=True)
#evaluator.plot_precision_vs_recall()

artf_name,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,7.089528,0.000000,0.923743,1.114650,2.150948,0.355476,4.012595,1.697277,3.342732,3.461319,1.227205,5.756607,7.164948,0.0
UC_007_TRG,9.834788,1.797495,1.810439,0.000000,0.742184,0.366688,3.204764,0.542561,0.742184,1.096859,9.115214,4.190159,0.330440,0.0
UC_010_TRG,16.810307,0.000000,6.353957,0.183335,1.333368,0.347976,0.916676,0.183335,2.280733,17.809227,0.183335,2.415047,1.469930,0.0
UC_002_TRG,5.330881,0.000000,0.000000,0.192850,1.533159,0.342094,2.128454,2.101317,1.219134,0.877039,0.342094,4.030716,2.237427,0.0
UC_006_TRG,5.080511,0.000000,0.000000,0.241239,0.368304,0.338581,2.452156,0.609543,1.711465,0.958172,0.368304,4.942810,2.290405,0.0
UC_004_TRG,3.906332,0.000000,0.000000,0.000000,1.524473,0.351135,3.682771,2.108979,1.053406,0.713617,0.351135,3.886018,5.459720,0.0
UC_005_TRG,5.319835,0.000000,0.000000,0.186401,0.371506,0.349795,2.502583,0.816893,1.704921,0.907702,0.371506,5.111546,2.367286,0.0
UC_008_TRG,13.300459,0.000000,0.000000,3.028716,9.086149,0.350704,6.057432,0.485578,6.057432,3.364868,0.000000,7.028083,31.087417,0.0
UC_001_TRG,2.346083,0.000000,1.031577,1.293069,1.135627,0.356966,3.094731,0.455406,2.949481,8.640988,0.000000,4.156527,4.604120,0.0


### Auxiliary Functions

In [ ]:
from itertools import product

def generate_params_comb_list(**kwargs):
    list_params = []
    for key, values in kwargs.items():
        aux_list = []
        for v in values:
            aux_list.append((key, v))
        list_params.append(aux_list)
    
    list_tuples = list(product(*list_params))
    
    list_dicts = []
    for ex_tup in list_tuples:
        dic = {}
        for in_tup in ex_tup:
            dic[in_tup[0]] = in_tup[1]
        list_dicts.append(dic)
        
    return list_dicts


def plot_heatmap(results_df):
    tmp_df = pd.DataFrame({'precision': results_df['precision'], 
                           'recall' : results_df['recall'], 
                           'fscore': results_df['fscore'], 
                           'model': results_df['model_name']})
    tmp_df.set_index('model', inplace=True)
    fig, ax = plt.subplots(figsize=(10, 4 * 10)) 
    ax = sns.heatmap(tmp_df, vmin=0, vmax=1, linewidths=.5, cmap="Greens", annot=True, cbar=False, ax=ax)


def highlight_df(df):
    cm = sns.light_palette("green", as_cmap=True)
    return df.style.background_gradient(cmap=cm)    

### Find The Best Model

In [120]:
all_hyperparams = {
    BM25_Model_Hyperp.TOP.value : [3,5],
    BM25_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('-', 0.0)],
    BM25_Model_Hyperp.TOKENIZER.value : [PorterStemmerBased_Tokenizer(), LancasterStemmerBased_Tokenizer(), 
                                                   WordNetBased_LemmaTokenizer(), SnowballStemmerBased_Tokenizer()]
}

hyperparams = generate_params_comb_list(**all_hyperparams)

print('Performing model hyperparameters search...')

def run_model(idx, **hyperp):    
    current_model = BM_25(**hyperp)
    current_model.set_name('BM25_Model_{}'.format(idx))
    current_model.recover_links(corpus, query, use_cases_names, bug_reports_names)
    
    evaluator = ModelEvaluator(orc.oracle, current_model)
    evaluator.evaluate_model()
    
    model_dump = 'dumps/model/{}.p'.format(evaluator.get_model().get_name())
    evaluator_dump = 'dumps/evaluator/eval_{}.p'.format(evaluator.get_model().get_name())
    
    pickle.dump(evaluator.get_model(), open(model_dump, 'wb'))
    pickle.dump(evaluator, open(evaluator_dump, 'wb'))
    
    return([evaluator.get_mean_precision(), 
                    evaluator.get_mean_recall(),
                    evaluator.get_mean_fscore(), 
                    evaluator.get_model().get_name(),
                    evaluator.get_model().get_top_value(),
                    evaluator.get_model().get_tokenizer_type(),
                    model_dump,
                    evaluator_dump
           ])

tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results_df = pd.DataFrame(data=results, 
                          columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'tokenizer', 'model_dump', 'evaluator_dump'])
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str})


Performing model hyperparameters search...


[Parallel(n_jobs=-1)]: Done   2 out of   8 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed:    0.2s finished


### Report

In [121]:
print("------------ Report -------------------\n")
print("Total of Analyzed Hyperparameters Combinations: {}".format(len(hyperparams)))

print("\nBest Model Hyperparameters Combination Found:\n")            

row_idx = results_df['model_dump'][results_df.recall == results_df.recall.max()].index[0]
best_model = pickle.load(open(results_df['model_dump'][row_idx], 'rb'))
evalu = pickle.load(open(results_df['evaluator_dump'][row_idx], 'rb'))
evalu.evaluate_model(verbose=True)

#print("\nPlot Precision vs Recall - Best Model")
#evalu.plot_precision_vs_recall()

#print("\nHeatmap of All Models")
#plot_heatmap(results_df)

#evalu.save_log()

------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 8

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of BM25_Model_6': 0.2859461966604825,
              'Mean Precision of BM25_Model_6': 0.17857142857142858,
              'Mean Recall of BM25_Model_6': 0.8095238095238095},
 'Setup': [{'Name': 'BM25_Model_6'},
           {'Top Value': 5},
           {'Sim Measure Min Threshold': ('-', 0.0)},
           {'K': 1.2},
           {'B': 0.75},
           {'Epsilon': 0.25},
           {'Tokenizer Type': <class '__main__.WordNetBased_LemmaTokenizer'>}]}


#### Best Model for TOP 3 and 5

In [125]:
for top in [3,5]:
    row_idx_top = results_df[results_df.top_value == top].recall.argmax()
    best_model_top = pickle.load(open(results_df['model_dump'][row_idx_top], 'rb'))
    evalu_top = pickle.load(open(results_df['evaluator_dump'][row_idx_top], 'rb'))
    evalu_top.evaluate_model(verbose=True)
    print("------------------------------------------------------------------")

{'Measures': {'Mean FScore of BM25_Model_2': 0.30346320346320343,
              'Mean Precision of BM25_Model_2': 0.2119047619047619,
              'Mean Recall of BM25_Model_2': 0.630952380952381},
 'Setup': [{'Name': 'BM25_Model_2'},
           {'Top Value': 3},
           {'Sim Measure Min Threshold': ('-', 0.0)},
           {'K': 1.2},
           {'B': 0.75},
           {'Epsilon': 0.25},
           {'Tokenizer Type': <class '__main__.WordNetBased_LemmaTokenizer'>}]}
------------------------------------------------------------------
{'Measures': {'Mean FScore of BM25_Model_6': 0.2859461966604825,
              'Mean Precision of BM25_Model_6': 0.17857142857142858,
              'Mean Recall of BM25_Model_6': 0.8095238095238095},
 'Setup': [{'Name': 'BM25_Model_6'},
           {'Top Value': 5},
           {'Sim Measure Min Threshold': ('-', 0.0)},
           {'K': 1.2},
           {'B': 0.75},
           {'Epsilon': 0.25},
           {'Tokenizer Type': <class '__main__.WordNetBased_

### Plot Highlights

In [122]:
highlight_df(best_model.get_trace_links_df())

artf_name,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,1,1,1,1,1,1,1,1,1,1,1,1,1,1
UC_007_TRG,1,1,1,0,0,1,1,0,0,0,1,0,0,1
UC_010_TRG,1,1,1,0,1,0,0,0,1,1,0,0,0,1
UC_002_TRG,0,1,0,1,1,0,0,1,0,0,0,0,0,1
UC_006_TRG,0,1,0,1,0,0,0,1,0,0,1,1,0,1
UC_004_TRG,0,1,0,0,1,1,1,1,0,0,0,0,1,1
UC_005_TRG,0,1,0,0,0,0,0,1,0,0,1,1,1,1
UC_008_TRG,1,1,0,1,1,1,1,0,1,1,0,1,1,1
UC_001_TRG,0,1,1,1,0,1,1,0,1,1,0,0,1,1


In [123]:
highlight_df(orc.oracle)

,BR_4020_SRC,BR_3890_SRC,BR_3844_SRC,BR_4065_SRC,BR_3880_SRC,BR_3987_SRC,BR_4067_SRC,BR_3973_SRC,BR_3898_SRC,BR_3908_SRC,BR_4058_SRC,BR_4018_SRC,BR_4005_SRC,BR_3974_SRC
artf_name,,,,,,,,,,,,,,
UC_003_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
UC_007_TRG,0,0,0,0,0,0,0,0,0,0,1,0,0,0
UC_010_TRG,0,0,1,0,0,0,0,0,0,0,0,0,0,0
UC_002_TRG,0,0,0,0,1,0,0,1,0,0,0,0,0,0
UC_006_TRG,1,1,0,1,1,0,0,0,0,0,0,1,0,1
UC_004_TRG,0,0,0,0,0,1,0,0,0,0,0,0,0,0
UC_005_TRG,1,1,0,0,1,0,1,0,0,0,0,1,0,0
UC_008_TRG,0,0,0,0,0,0,0,0,0,0,0,0,1,0
UC_001_TRG,0,0,0,0,0,0,0,0,0,0,0,0,0,0
